In [27]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def generate_interferogram(S, Er_cases, Eis):
    lambda_ = 532e-9  # Longitud de onda en metros (532 nm)
    k = 2 * np.pi / lambda_  # Número de onda
    x = np.linspace(-k, k, 500)  # Coordenada espacial
    mu = 0  # Desfase global

    interferograms = {}

    for Er_label, Er in Er_cases.items():
        Es = S @ Eis  # Campo eléctrico transmitido a través del objeto
        I = np.abs(Er[0])**2 + np.abs(Er[1])**2 + np.abs(Es[0])**2 + np.abs(Es[1])**2
        
        # Definición de parámetros \(\Omega\), \(\Psi\) y \(\zeta\)
        Omega = np.abs(Er[0])**2 + np.abs(Es[0])**2 + np.abs(Es[1])**2
        Psi = np.abs(Er[0]) * np.abs(Es[0])
        zeta = np.mean(k * x)  # Usar un único valor promedio para zeta

        # Cálculo de la distribución de intensidad del patrón de interferencia
        I_pattern = Omega + Psi * np.cos(mu + k * x - zeta)

        # Normalización de la intensidad para escalar entre 0 y 255 (imagen en escala de grises)
        I_pattern_norm = (I_pattern - I_pattern.min()) / (I_pattern.max() - I_pattern.min()) * 255
        I_pattern_norm = np.uint8(I_pattern_norm)

        # Crear imagen de patrón de interferencia en escala de grises
        height = 500  # Altura de la imagen en píxeles
        pattern_image = np.tile(I_pattern_norm, (height, 1))

        # Aplicar un filtro Gaussiano para suavizar las franjas
        pattern_image = cv2.GaussianBlur(pattern_image, (5, 5), 0)

        interferograms[Er_label] = (pattern_image, float(Omega), float(Psi), float(zeta), np.abs(Es[1]))

    return interferograms

def calculate_birefringence(Omega1, Omega2, Psi1, Psi2, zeta1, zeta2, Es1_y, Es2_y, Er_x, v2):
    numerator = ((Omega1 + Omega2) * Er_x * Psi1) * np.tan(v2)
    denominator = (Omega1 + Omega2) + 2 * Er_x * Psi1 * Er_x * Es2_y
    Delta_chi = np.arctan2(numerator, denominator)
    return Delta_chi*2

# Definición de la matriz S basada en la teoría de birrefringencia
Delta_gamma = np.pi / 4  # Retardo de fase arbitrario
n = 532e-9/(1e-7 * 2 * np.pi) * Delta_gamma
alpha = np.pi / 6  # Ángulo del eje óptico
S = np.array([[np.cos(alpha)**2 * np.exp(1j * Delta_gamma / 2) + np.sin(alpha)**2 * np.exp(-1j * Delta_gamma / 2), 
               2j * np.cos(alpha) * np.sin(alpha) * np.sin(Delta_gamma / 2)],
              [2j * np.cos(alpha) * np.sin(alpha) * np.sin(Delta_gamma / 2),
               np.sin(alpha)**2 * np.exp(1j * Delta_gamma / 2) + np.cos(alpha)**2 * np.exp(-1j * Delta_gamma / 2)]])

# Definición de los casos de Er
Er_cases = {
    "Er_1_0": np.array([1, 0]),
    "Er_1_1": np.array([1, 1]),
    "Er_1_1_Eis_1_1": np.array([1, 1])  # Este será usado con Eis = [1,1]
}

# Definición de Eis con una polarización elíptica arbitraria
Eis_elliptical = np.array([1, np.exp(1j * np.pi / 8)])

# Generar interferogramas para los primeros dos casos con Eis arbitrario
interferograms = generate_interferogram(S, Er_cases, Eis_elliptical)

# Generar interferograma para el tercer caso con Eis = [1,1]
Eis_linear = np.array([1, 1])
interferograms["Er_1_1_Eis_1_1"] = generate_interferogram(S, {"Er_1_1_Eis_1_1": np.array([1, 1])}, Eis_linear)["Er_1_1_Eis_1_1"]

# Cálculo de la birrefringencia usando la ecuación de \(\Delta \chi\)
Omega1 = interferograms["Er_1_0"][1]
Omega2 = interferograms["Er_1_1"][1]
Psi1 = interferograms["Er_1_0"][2]
Psi2 = interferograms["Er_1_1"][2]
zeta1 = interferograms["Er_1_0"][3]
zeta2 = interferograms["Er_1_1"][3]
Es1_y = interferograms["Er_1_0"][4]
Es2_y = interferograms["Er_1_1"][4]
Er_x = 1  # Componente x de Er
v2 = np.pi / 8  # Ángulo arbitrario

Delta_chi = calculate_birefringence(Omega1, Omega2, Psi1, Psi2, zeta1, zeta2, Es1_y, Es2_y, Er_x, v2)

# Mostrar la birrefringencia calculada y el error relativo
error_relativo = abs((Delta_chi - n) / n) if not np.isnan(Delta_chi) else np.nan
print("Birefringencia colocada en el objeto:", n)
print("Birefringencia calculada:", Delta_chi)
print("Error relativo:", error_relativo)

Birefringencia colocada en el objeto: 0.665
Birefringencia calculada: 0.5689330658406702
Error relativo: 0.14446155512681183
